In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/kitti_raw_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/image_02/data/0000000093.png  
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/image_02/data/0000000094.png  
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/image_02/data/0000000095.png  
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/image_02/data/0000000097.png  
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/image_02/data/0000000096.png  
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/image_02/data/0000000098.png  
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/image_02/data/0000000100.png  
  inflating: content/drive/MyDrive/kitti_raw_data/2011_09_26/2011_09_26_drive_0086_extract/imag

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2

In [ ]:
Resnet = ResNet50V2(include_top=False, weights=None,
    input_shape=(256,256,3),
    pooling=None)

In [ ]:
Resnet.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 130, 130, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential

layer_name = 'conv4_block6_out'
layer_output = Resnet.get_layer(name = layer_name).output
intermediate_model=tf.keras.models.Model(inputs=Resnet.input,outputs=layer_output)
backbone = Sequential()
backbone.add(intermediate_model)

In [ ]:
backbone.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 8, 8, 1024)        8573440   
Total params: 8,573,440
Trainable params: 8,548,480
Non-trainable params: 24,960
_________________________________________________________________


In [ ]:
from tensorflow.keras.activations import tanh
from tensorflow.keras.layers import Conv2DTranspose, BatchNormalization, Dropout, ReLU, add, Flatten, Activation

def upsample(filters, size, stride, shape, apply_dropout = False, last_layer=False):

    initializer = tf.random_normal_initializer(0.,0.02)

    result = Sequential()

    result.add(Conv2DTranspose(filters,size,strides=stride,padding='same',batch_input_shape=shape,kernel_initializer=initializer, use_bias=False))

    result.add(BatchNormalization())

    if apply_dropout:

        result.add(Dropout(0.5))

    if last_layer:
      result.add(Activation('tanh'))
      print("yay last layer")
    else:
      result.add(ReLU())

    return result

In [ ]:
head1 = Sequential()

head1.add(backbone)
head1.add(upsample(256,4,1,(None,8,8,1024),apply_dropout=True))
head1.add(upsample(128,4,2,(None,8,8,256),apply_dropout=True))
head1.add(upsample(64,4,1,(None,16,16,128)))
head1.add(upsample(32,4,2,(None,16,16,64)))
head1.add(upsample(1,4,1,(None,32,32,32),last_layer=True))
head1.add(Flatten())

yay last layer


In [ ]:
head1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 8, 8, 1024)        8573440   
_________________________________________________________________
sequential_2 (Sequential)    (None, 8, 8, 256)         4195328   
_________________________________________________________________
sequential_3 (Sequential)    (None, 16, 16, 128)       524800    
_________________________________________________________________
sequential_4 (Sequential)    (None, 16, 16, 64)        131328    
_________________________________________________________________
sequential_5 (Sequential)    (None, 32, 32, 32)        32896     
_________________________________________________________________
sequential_6 (Sequential)    (None, 32, 32, 1)         516       
_________________________________________________________________
flatten (Flatten)            (None, 1024)             

In [ ]:
head2 = Sequential()

head2.add(backbone)
head2.add(upsample(256,4,1,(None,8,8,1024),apply_dropout=True))
head2.add(upsample(128,4,2,(None,8,8,256),apply_dropout=True))
head2.add(upsample(64,4,1,(None,16,16,128)))
head2.add(upsample(32,4,2,(None,16,16,64)))
head2.add(upsample(1,4,1,(None,32,32,32), last_layer=True))
head2.add(Flatten())

yay last layer


In [ ]:
head2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 8, 8, 1024)        8573440   
_________________________________________________________________
sequential_8 (Sequential)    (None, 8, 8, 256)         4195328   
_________________________________________________________________
sequential_9 (Sequential)    (None, 16, 16, 128)       524800    
_________________________________________________________________
sequential_10 (Sequential)   (None, 16, 16, 64)        131328    
_________________________________________________________________
sequential_11 (Sequential)   (None, 32, 32, 32)        32896     
_________________________________________________________________
sequential_12 (Sequential)   (None, 32, 32, 1)         516       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)             

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

class DistanceLayer(layers.Layer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):

        anchor = anchor/tf.norm(anchor,axis=-1, keepdims = True)
        positive = positive/tf.norm(positive,axis=-1, keepdims = True)
        negative = negative/tf.norm(negative,axis=-1, keepdims = True)
    
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)

anchor_input = layers.Input(name="anchor", shape=(256,256,3,))
positive_input = layers.Input(name="positive", shape=(256,256,3,))
negative_input = layers.Input(name="negative", shape=(256,256,3,))


In [ ]:
distances1 = DistanceLayer()(
    (head1(anchor_input)),
    (head1(positive_input)),
    (head1(negative_input)),
)

siamese_network1 = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances1
)

#Make siamese_network

In [ ]:
siamese_network1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor (InputLayer)             [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
positive (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
negative (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1024)         13458308    anchor[0][0]                     
                                                                 positive[0][0]             

In [ ]:
distances2 = DistanceLayer()(
    (head2(anchor_input)),
    (head2(positive_input)),
    (head2(negative_input)),
)

siamese_network2 = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances2
)
#Make siamese_network

In [ ]:
def compute_loss(ap_distance, an_distance, margin):
      
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + margin, 0.0)

        return loss

#Siamese Loss

In [ ]:
siamese1_optimizer = tf.keras.optimizers.Adam(2e-6,beta_1=0.5)
siamese2_optimizer = tf.keras.optimizers.Adam(2e-6, beta_1=0.5)

In [ ]:
folders = ['/content/content/drive/MyDrive/kitti_raw_data/2011_09_26']

In [ ]:
import os
imgs_path = []
for folder in folders:
  subfolders = os.listdir(folder)
  for subfolder in subfolders:
    subfolder_path = os.path.join(folder,subfolder)
    extra_path = 'image_02/data'
    final_path = os.path.join(subfolder_path, extra_path)
    images = os.listdir(final_path)
    for img in images:
      img_path = os.path.join(final_path,img)
      imgs_path.append(img_path)

# Add path of all images to a list

In [ ]:
print(len(imgs_path))

14465


In [ ]:
def train(X1_anchor, X1_pos, X1_neg, X2_anchor, X2_pos, X2_neg):

  with tf.GradientTape() as s1_tape, tf.GradientTape() as s2_tape:

        ap1, an1 = siamese_network1([X1_anchor, X1_pos, X1_neg], training = True)
        ap2, an2 = siamese_network2([X2_anchor, X2_pos, X2_neg], training = True)

        loss1 = compute_loss(ap1, an1, margin= 0.6)

        loss2 = compute_loss(ap2 ,an2, margin = 0.3)

  s1_gradients = s1_tape.gradient(loss1,siamese_network1.trainable_variables)
  s2_gradients = s2_tape.gradient(loss2,siamese_network2.trainable_variables)

  siamese1_optimizer.apply_gradients(zip(s1_gradients,siamese_network1.trainable_variables))
  siamese2_optimizer.apply_gradients((zip(s2_gradients,siamese_network2.trainable_variables)))

  return loss1, loss2

#Train function

In [ ]:
#1 = Background
#2 = Foreground

previous_epoch = 50

import random
import numpy as np 
import cv2

epochs = 100
batch_size = 8

X1_anchor = np.zeros((batch_size, 256, 256,3))
X1_pos = np.zeros((batch_size, 256, 256,3))
X1_neg = np.zeros((batch_size, 256, 256,3))

X2_anchor = np.zeros((batch_size, 256, 256,3))
X2_pos = np.zeros((batch_size, 256, 256,3))
X2_neg = np.zeros((batch_size, 256, 256,3))

total_loss1 = 0
total_loss2 = 0

batch_count = 0

checkpoint_dirs = '/content/drive/MyDrive/SSL_small'
checkpoint_prefix = os.path.join(checkpoint_dirs,"ckpt")
checkpoint = tf.train.Checkpoint(siamese1_optimizer = siamese1_optimizer, siamese2_optimizer = siamese1_optimizer, siamese_network1 = siamese_network1, siamese_network2 = siamese_network2)

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dirs))

for epoch in range(previous_epoch,epochs):
  
  random.shuffle(imgs_path)
  count = 0
  batch_count = 0
  total_loss1 = 0
  total_loss2 = 0

#img1 - current image
#img2 - next image
#img3 - next image after img2
#img4 - Some random image which is not of current folder

  for img in imgs_path:
    num = int(img[-7:-4])
    next_path = img[:-7]
    next_num = str(num+1)
    if len(next_num) == 1:
      next_num = '00' + next_num
    elif len(next_num) ==2:
       next_num = '0' + next_num
    next_path = next_path + next_num + '.png'
    img1 = cv2.imread(img)
    img2 = cv2.imread(next_path)
    next_path = img[:-7]
    next_num = str(num+2)
    if len(next_num) == 1:
      next_num = '00' + next_num
    elif len(next_num) ==2:
       next_num = '0' + next_num
    next_path = next_path + next_num + '.png'
    img3 = cv2.imread(next_path)
    img4_path = random.choice(imgs_path)
    #if img4_path[49:70] == img[49:70]:
    if img4_path[70:80] == img[70:80]:
      continue
    img4 = cv2.imread(img4_path)

  
    if img2 is not None and img3 is not None and img4 is not None :
      img1 = cv2.resize(img1, (256,256))
      img2 = cv2.resize(img2, (256,256))
      img3 = cv2.resize(img3, (256,256))
      img4 = cv2.resize(img4, (256,256))
      
      X1_anchor[count] = img1/256
      X1_pos[count] = img3/256
      X1_neg[count] = img4/256

      X2_anchor[count] = img1/256
      X2_pos[count] = img2/256
      X2_neg[count] = img3/256

      count = count + 1

      if count == batch_size:
        #ap1, an1 = siamese_network1([X1_anchor, X1_pos, X1_neg], training = True)
        loss1, loss2 = train(X1_anchor, X1_pos, X1_neg, X2_anchor, X2_pos, X2_neg)
        total_loss1 = total_loss1 + tf.math.reduce_mean(loss1)
        total_loss2 = total_loss2 + tf.math.reduce_mean(loss2)
        count = 0
        batch_count = batch_count + 1
        if batch_count%100==0:
          print(batch_count)
          print(total_loss1/batch_count)
          print(total_loss2/batch_count)

  f = open('/content/drive/MyDrive/SSL_small/report.txt', 'a')
  
  f.write("\n")
  f.write('epoch: ')
  f.write(str(epoch))
  f.write("\n")
  f.write('total_loss1: ')
  f.write(str(total_loss1/batch_count))
  f.write("\n")
  f.write('total_loss2: ')
  f.write(str(total_loss2/batch_count))

  print("Epoch ", epoch , "done")
  checkpoint.save(file_prefix=checkpoint_prefix)
  head1.save_weights("/content/drive/MyDrive/SSL_small/head1_{}.h5".format(str(epoch)))
  head2.save_weights("/content/drive/MyDrive/SSL_small/head2_{}.h5".format(str(epoch)))


100
tf.Tensor(0.00048160873, shape=(), dtype=float32)
tf.Tensor(0.034149468, shape=(), dtype=float32)
200
tf.Tensor(0.00068432273, shape=(), dtype=float32)
tf.Tensor(0.02909489, shape=(), dtype=float32)
300
tf.Tensor(0.00052308425, shape=(), dtype=float32)
tf.Tensor(0.030173864, shape=(), dtype=float32)
400
tf.Tensor(0.0004484126, shape=(), dtype=float32)
tf.Tensor(0.029208686, shape=(), dtype=float32)
500
tf.Tensor(0.00041466363, shape=(), dtype=float32)
tf.Tensor(0.029300304, shape=(), dtype=float32)
600
tf.Tensor(0.00040138626, shape=(), dtype=float32)
tf.Tensor(0.029791301, shape=(), dtype=float32)
700
tf.Tensor(0.0004138721, shape=(), dtype=float32)
tf.Tensor(0.029793905, shape=(), dtype=float32)
800
tf.Tensor(0.00042825937, shape=(), dtype=float32)
tf.Tensor(0.030442206, shape=(), dtype=float32)
900
tf.Tensor(0.00043377728, shape=(), dtype=float32)
tf.Tensor(0.030435702, shape=(), dtype=float32)
1000
tf.Tensor(0.00048394167, shape=(), dtype=float32)
tf.Tensor(0.030911459, shape=(

In [ ]:
sadas